In [ ]:
import cv2
import matplotlib
import numpy

##Mount your google drive to colab server

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Create a folder name Colab Notebooks in your google drive....this is where your downloaded darknet framework will be stored

In [ ]:
cd /content/drive/My Drive/Colab Notebooks/

clone the darknet framework from the github repo as follows

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

cd to the darknet folder inside the Colab Notebooks folder you created above

In [ ]:
cd /content/drive/My Drive/Colab Notebooks/darknet

### Compile darknet

In [ ]:
%cd darknet
!sed -i 's/CUDNN=0/CUDNN=0/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!cat Makefile
!make

#Download your custom data set prepared in the format used for training YOLOV3 model


In [ ]:

import os.path
import shutil
from google.colab import drive

DOWNLOAD_LOCATION = '/content/drive/My Drive/Colab Notebooks/darknet/data/' # create the folders in your G-drive as indicated here
DRIVE_DATASET_FILE = '/content/drive/My Drive/Colab Notebooks/ml/xxxxx.rar' #adjust path/name of dataset which is in your G-drive

shutil.copy(DRIVE_DATASET_FILE, DOWNLOAD_LOCATION)

print('Successfully downloaded the dataset')


### Unzip the dataset

In [ ]:
!unzip  /content/drive/My\ Drive/Colab\ Notebooks/darknet/data/xxxxx.rar -d /content/drive/My\ Drive/Colab\ Notebooks/darknet/data/ # adjust the dataset filename which you have downloaded from Google drive


### if your dataset is small use pretrained weight file which can be downloaded as follows. If you want to train your model from scrtach using your custom dataset you can jump this step

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

### Train your model using your custom dataset. If you are not well aware of the steps how to train and how to set hyperparameters when training yolov3 model using custom dataset read furthe on the github repo of AlexyAB's site https://github.com/AlexeyAB/darknet



In [ ]:
%cd darknet
!./darknet detector train data/trainer.data cfg/yolov3.cfg darknet53.conv.74  -dont_show -map | tee backup/log.txt 

## You can test your model prediction as follows on sample test image. thresh and iou_thresh are optional you can change as you want

In [ ]:
!./darknet detector test data/trainer.data cfg/yolov3.cfg backup/yolov3_last.weights -thresh 0.1  -iou_thresh 0.3  data/img/xxxxxx.jpg

#once you decided your model predicts well on your problem domain now it is time to convert or to deploy the model in to your android device. First download clone the folowing github repo to your google drive. This repo helps you to convert the trained yolov3 model to .pb file which is used further processing in the tensorflow android API. run the cells step by step

In [ ]:
!git clone https://github.com/jinyu121/DW2TF

In [ ]:
!unzip DW2TF-master.zip

In [ ]:
!python3 main.py \
  --cfg 'data/yolov3.cfg' \
  --weights 'data/yolov3_last.weights' \
  --output 'data/' \
  --prefix 'yolov3/' \
  --gpu 0

#getting input and output node names from yolov3 .pb file is difficult. if you get difficulty on it use this blog post https://medium.com/@afetulhak/how-to-find-input-and-output-node-names-for-yolov3-from-tensorflow-pb-file-using-google-colab-2dc0f9e73258

In [ ]:
!python3 freeze_graph.py  \
 --input_graph data/yolov3.pb  \
 --input_checkpoint data/yolov3.ckpt  \
 --input_binary=true  \
 --output_graph=ultimate_yolov3.bp  \
 --output_node_names=yolov3/convolutional59/BiasAdd,yolov3/convolutional67/BiasAdd,yolov3/convolutional75/BiasAdd

After running all the cells you will get .bp file in your google drive. Download this file and put it in asset folder of the android studio code